In [13]:
import numpy as np
import random as rd
import math

In [14]:
orientation_list = [ [[1,0],[0,1]], [[0,1],[-1,0]], [[-1,0],[0,-1]], [[0,-1],[1,0]] ] 
shift_dict = { "L": [[0,0], [1,0], [1,-1], [0,-1]], "D": [[0,0], [2,0], [1,-1], [0,-1]]}

def transform_hole(hole, orientation, piece):    
    matrix = np.array(orientation_list[orientation])
    vector = np.array(list(hole))
    rotated = np.matmul(matrix, vector)
    shift_array = np.array(shift_dict[piece][orientation])
    return list(rotated + shift_array)

def draw(l):
    size = len(l)
    s = ""
    s+="-"*(4*size+1)
    s+="\n"
    for i in range(size):
        s+="|"
        for j in range(size):
            s+=" "
            s+=str(l[i][j])
            s+=" |"
        s+="\n"
        s+="-"*(4*size+1)
        s+="\n"
    return s

class block:
    def __init__(self, piece, height, width, tall, orientation=1, holes=[]):
        self.piece = piece
        self.height = height
        self.width = width
        self.tall = tall
        self.holes = holes #list of holes, each hole is a list of [y,x]
        self.orientation = orientation #num of possible oreintation

    def area(self):
        return self.height*self.width
    
def setup():
    # pieces
    global L, T, I, F
    L = block("L",2,2,2,4,[[0,1]])
    T = block("T",2,3,2,4,[[0,1],[2,1]])
    I = block("I",2,1,2,2,[])
    F = block("F",2,2,2,1,[])
    #  constants
    global threshold
    threshold = 50

class slot:
    def __init__(self, piece='d'):
        self.piece = piece  

    def change_piece(self, new_piece):
        self.piece = new_piece

def generate_board(board_size):
    board_ = []
    for i in range(board_size):
        board_.append([slot() for i in range(board_size)])
    return board_

def transform(array):
    l = []
    for i in array:
        temp = []
        for j in i:
            temp.append(slot(j))
        l.append(temp)
    return np.array(l)


class game_board:
    def __init__(self, auto=True, array=np.array([]), size=4):
        if not auto:
            self.board = transform(array)
        else:
            self.board = generate_board(size)
        self.size=size

    def __str__(self):
        view = []
        for row in self.board:
            temp = []
            for obj in row:
                temp.append(obj.piece)
            view.append(temp)
        return draw(view)
    
    def clear(self):
        self.board = generate_board(self.size)
    
    def set_piece(self, row, col, piece):
        self.board[row][col].change_piece(piece)

    def check_fit(self, height, width, row, col, hole=[]):
        true_hole = [(i+row, j+col) for i,j in hole]
        for y in range(height):
            for x in range(width):
                if row+y>=self.size or col+x>=self.size:
                    return False
                else:
                    if (row+y, col+x) not in true_hole:
                        if self.board[row+y][col+x].piece != 'd':
                            return False
        return True

    def place_block(self, height, width, row, col, piece, hole=[]):
        true_hole = [(i+row, j+col) for i,j in hole]
        for y in range(height):
            for x in range(width):
                if (row+y, col+x) not in true_hole and row+y<self.size and col+x<self.size:
                    self.set_piece(row+y,col+x,piece)

    def randomize(self, piece: block):
        r = rd.randint(0,self.size)
        c = rd.randint(0,self.size)
        o = rd.randint(0,piece.orientation-1)
        holes = []
        for old_hole in piece.holes:
            holes.append(transform_hole(old_hole, o, piece.piece))
        if self.check_fit(piece.height, piece.width, r, c, holes):
            self.place_block(piece.height, piece.width, r, c, piece.piece, holes)
            print(f"randomize() --> success (r,c,o) = ({r},{c},{o})")
            return True
        else:
            print(f"randomize() --> Failed (r,c,o) = ({r},{c},{o})")
            return False

    def randomize_pieces(self, pieces):
        new_list = sorted([[piece.area()-len(piece.holes), piece.piece, piece] for piece in pieces], reverse=True)
        for area, name, piece in new_list:
            for i in range(threshold):
                if self.randomize(piece):
                    print(f"randomize_pieces --> success {piece.piece}")
                    break   

Playground

In [31]:
main = game_board()
setup()

In [34]:
print(main)

-----------------
| d | L | L | d |
-----------------
| d | L | L | d |
-----------------
| F | F | I | d |
-----------------
| F | F | I | d |
-----------------



In [50]:
main.randomize(I)
print(main)

randomize() --> success (r,c,o) = (0,3,1)
-----------------
| d | L | L | I |
-----------------
| d | L | L | I |
-----------------
| F | F | I | d |
-----------------
| F | F | I | d |
-----------------



In [33]:
main.randomize_pieces([L, F, I])

randomize() --> Failed (r,c,o) = (4,3,1)
randomize() --> Failed (r,c,o) = (4,2,0)
randomize() --> Failed (r,c,o) = (4,1,2)
randomize() --> success (r,c,o) = (0,1,1)
randomize_pieces --> success L
randomize() --> Failed (r,c,o) = (4,1,0)
randomize() --> Failed (r,c,o) = (4,3,0)
randomize() --> Failed (r,c,o) = (1,3,0)
randomize() --> Failed (r,c,o) = (0,1,0)
randomize() --> Failed (r,c,o) = (1,0,0)
randomize() --> Failed (r,c,o) = (4,0,0)
randomize() --> Failed (r,c,o) = (2,3,0)
randomize() --> Failed (r,c,o) = (0,2,0)
randomize() --> Failed (r,c,o) = (4,0,0)
randomize() --> Failed (r,c,o) = (0,2,0)
randomize() --> Failed (r,c,o) = (4,3,0)
randomize() --> Failed (r,c,o) = (1,3,0)
randomize() --> Failed (r,c,o) = (1,3,0)
randomize() --> Failed (r,c,o) = (1,1,0)
randomize() --> Failed (r,c,o) = (4,2,0)
randomize() --> Failed (r,c,o) = (3,4,0)
randomize() --> Failed (r,c,o) = (0,4,0)
randomize() --> Failed (r,c,o) = (0,1,0)
randomize() --> Failed (r,c,o) = (0,3,0)
randomize() --> success (

In [51]:
main.clear()

In [19]:
for i in range(4):
    print()
    for x in range(2):
        print("".join(["▣" if transform_hole([0,0],i,"L") != [x,y] else " " for y in range(-1,1)]))


▣ 
▣▣

▣▣
▣ 

▣▣
 ▣

 ▣
▣▣


In [20]:
L = block("L",2,2,2,[[1,0]],0)
R = block("R",2,1,2,[[1,0]],0)

pieces = [L, R]
new_list = sorted([[piece.area(), piece] for piece in pieces],reverse=True)
print(new_list)

[[4, <__main__.block object at 0x000001F38A50BB50>], [2, <__main__.block object at 0x000001F38A50BA60>]]


In [21]:
setup()

In [22]:
L.piece

'L'

In [23]:
main = game_board()

In [24]:
print(main)

-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------



In [25]:
main.place_block(2,2,2,2,'f', hole=[(0,1)])

In [26]:
main.check_fit(1,1,2,2)

False

In [27]:
print(main)

-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------
| d | d | f | d |
-----------------
| d | d | f | f |
-----------------



In [28]:
main.clear()

In [29]:
print(main)

-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------
| d | d | d | d |
-----------------

